In [110]:
from icepyx import icesat2data as ipd
import os
import shutil
from pathlib import Path
from pprint import pprint

import pandas as pd

%matplotlib inline

In [2]:
short_name = 'ATL06'
spatial_extent = [ -50.71, -45.32, 68.08, 65.73]
date_range = ['2019-06-01','2019-06-05']

In [3]:
region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range)

print(region_a.dataset)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.dataset_version)
print(region_a.spatial_extent)

ATL06
['2019-06-01', '2019-06-05']
00:00:00
23:59:59
003
['bounding box', [-50.71, -45.32, 68.08, 65.73]]


In [4]:
region_a.avail_granules(ids=True)

['ATL06_20190601005859_09740303_003_01.h5',
 'ATL06_20190601023316_09750303_003_01.h5',
 'ATL06_20190601035904_09760302_003_01.h5',
 'ATL06_20190601054151_09770303_003_01.h5',
 'ATL06_20190601071608_09780303_003_01.h5',
 'ATL06_20190601120935_09810305_003_01.h5',
 'ATL06_20190601121500_09810306_003_01.h5',
 'ATL06_20190601134352_09820305_003_01.h5',
 'ATL06_20190601134917_09820306_003_01.h5',
 'ATL06_20190601140450_09820308_003_01.h5',
 'ATL06_20190601151810_09830305_003_01.h5',
 'ATL06_20190601182644_09850305_003_01.h5',
 'ATL06_20190601200102_09860305_003_01.h5',
 'ATL06_20190602000443_09880313_003_01.h5',
 'ATL06_20190602002449_09890302_003_01.h5',
 'ATL06_20190602003320_09890303_003_01.h5',
 'ATL06_20190602020737_09900303_003_01.h5',
 'ATL06_20190602033324_09910302_003_01.h5',
 'ATL06_20190602034154_09910303_003_01.h5',
 'ATL06_20190602065029_09930303_003_01.h5',
 'ATL06_20190602114355_09960305_003_01.h5',
 'ATL06_20190602114920_09960306_003_01.h5',
 'ATL06_20190602131813_09970305_

In [6]:
earthdata_emails = {'tsnow03':'tasha.snow@colorado.edu',
                 'fperez': 'fernando.perez@berkeley.edu',
                 # add your name here
                }

user = 'fperez'
region_a.earthdata_login(user, earthdata_emails[user])

Earthdata Login password:  ··········


In [7]:
region_a.order_vars.avail(options=True)

var_list inputs: atlas_sdp_gps_epoch, control, data_end_utc, data_start_utc, end_cycle, end_delta_time, end_geoseg, end_gpssow, end_gpsweek, end_orbit, end_region, end_rgt, granule_end_utc, granule_start_utc, qa_at_interval, release, start_cycle, start_delta_time, start_geoseg, start_gpssow, start_gpsweek, start_orbit, start_region, start_rgt, version, dt_hist, fit_maxiter, fpb_maxiter, maxiter, max_res_ids, min_dist, min_gain_th, min_n_pe, min_n_sel, min_signal_conf, n_hist, nhist_bins, n_sigmas, proc_interval, qs_lim_bsc, qs_lim_hrs, qs_lim_hsigma, qs_lim_msw, qs_lim_snr, qs_lim_sss, rbin_width, sigma_beam, sigma_tx, t_dead, atl06_quality_summary, delta_time, h_li, h_li_sigma, latitude, longitude, segment_id, sigma_geo_h, fpb_mean_corr, fpb_mean_corr_sigma, fpb_med_corr, fpb_med_corr_sigma, fpb_n_corr, med_r_fit, tx_mean_corr, tx_med_corr, dem_flag, dem_h, geoid_h, dh_fit_dx, dh_fit_dx_sigma, dh_fit_dy, h_expected_rms, h_mean, h_rms_misfit, h_robust_sprd, n_fit_photons, n_seg_pulses,

In [9]:
region_a.order_vars.remove(all=True)
region_a.order_vars.append(defaults=True)
from IPython.display import JSON
JSON(region_a.order_vars.wanted)

<IPython.core.display.JSON object>

In [10]:
# region_a.order_vars.append(beam_list=['gt1l', 'gt2l', 'gt3l'],
#                            var_list=['longitude','latitude','atl06_quality_summary',
#                                      'cloud_flg_asr','cloud_flg_atm','msw_flag',
#                                      'bsnow_conf','bsnow_h','bsnow_od'])
# pprint(region_a.order_vars.wanted)

Now, specify our coverage request with the wanted variables:

In [15]:
region_a.subsetparams(Coverage=region_a.order_vars.wanted);

In [16]:
region_a.order_granules()

Total number of data order requests is  6  for  58  granules.
Data request  1  of  6  is submitting to NSIDC
order ID:  5000000699973
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is:  complete_with_errors
NSIDC provided these error messages:
['178076054:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178076053:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178076743:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178135806:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '178076741:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 'PT5.174S',
 'ICESAT2']
Your order is: complete_with_errors
Data request  2  of  6  is submitting to NSIDC
order ID:  5000000699974
Initial status o

In [17]:
# View a short list of order IDs:
region_a.granules.orderIDs

['5000000699973',
 '5000000699974',
 '5000000699975',
 '5000000699976',
 '5000000699977',
 '5000000699978']

In [18]:
path = './download'
# without variable subsetting, or with variable subsetting if you have run region_a.order_granules(Coverage=region_a.order_vars.wanted)
region_a.download_granules(path)

Beginning download of zipped output...
Data request 5000000699973 of  6  order(s) is downloaded.
Beginning download of zipped output...
Data request 5000000699974 of  6  order(s) is downloaded.
Beginning download of zipped output...
Data request 5000000699975 of  6  order(s) is downloaded.
Beginning download of zipped output...
Data request 5000000699976 of  6  order(s) is downloaded.
Beginning download of zipped output...
Data request 5000000699977 of  6  order(s) is downloaded.
Beginning download of zipped output...
Data request 5000000699978 of  6  order(s) is downloaded.
Download complete


Some utility functions

In [19]:
import pyproj
from astropy.time import Time

def gps2dyr(time):
    """Converte GPS time to decimal years."""
    return Time(time, format='gps').decimalyear


def orbit_type(time, lat, tmax=1):
    """Separate tracks into ascending and descending.
    
    Defines tracks as segments with time breaks > tmax,
    and tests whether lat increases or decreases w/time.
    """
    tracks = np.zeros(lat.shape)  # generate track segment
    tracks[0:np.argmax(np.abs(lat))] = 1  # set values for segment
    is_asc = np.zeros(tracks.shape, dtype=bool)  # output index array

    # Loop trough individual secments
    for track in np.unique(tracks):
    
        i_track, = np.where(track == tracks)  # get all pts from seg
    
        if len(i_track) < 2: continue
    
        # Test if lat increases (asc) or decreases (des) w/time
        i_min = time[i_track].argmin()
        i_max = time[i_track].argmax()
        lat_diff = lat[i_track][i_max] - lat[i_track][i_min]
    
        # Determine track type
        if lat_diff > 0:  is_asc[i_track] = True
    
    return is_asc


def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projections:
        Geodetic (lon/lat): 4326
        Polar Stereo AnIS (x/y): 3031
        Polar Stereo GrIS (x/y): 3413
    """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert


In [100]:
import h5py
import numpy as np

def read_atl06(fname, outdir='data', bbox=None):
    """Read one ATL06 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """

    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
    
    # Loop trough beams
    dataframes = []  # one dataframe per track
    
    with h5py.File(fname, 'r') as fi:
        # Check which ground tracks are present in this file
        gtracks = sorted(['/'+k for k in fi.keys() if k.startswith('gt')])
        print('Available ground tracks:', gtracks)
    
        for k, g in enumerate(gtracks): 
            #-----------------------------------#
            # 1) Read in data for a single beam #
            #-----------------------------------#
            data = {}
            data['ground_track'] = None # Put it first in the dict for column ordering
    
            # Load vars into memory (include as many as you want)
            data['lat'] = fi[g+'/land_ice_segments/latitude'][:]
            data['lon'] = fi[g+'/land_ice_segments/longitude'][:]
            data['h_li'] = fi[g+'/land_ice_segments/h_li'][:]
            data['s_li'] = fi[g+'/land_ice_segments/h_li_sigma'][:]
            data['t_dt'] = fi[g+'/land_ice_segments/delta_time'][:]
            data['q_flag'] = fi[g+'/land_ice_segments/atl06_quality_summary'][:]
#                 data['s_fg'] = fi[g+'/land_ice_segments/fit_statistics/signal_selection_source'][:]
#                 data['snr'] = fi[g+'/land_ice_segments/fit_statistics/snr_significance'][:]
#                 data['h_rb'] = fi[g+'/land_ice_segments/fit_statistics/h_robust_sprd'][:]
#                 data['dac'] = fi[g+'/land_ice_segments/geophysical/dac'][:]
#                 data['f_sn'] = fi[g+'/land_ice_segments/geophysical/bsnow_conf'][:]
#                 data['dh_fit_dx'] = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dx'][:]
#                 data['tide_earth'] = fi[g+'/land_ice_segments/geophysical/tide_earth'][:]
#                 data['tide_load'] = fi[g+'/land_ice_segments/geophysical/tide_load'][:]
#                 data['tide_ocean'] = fi[g+'/land_ice_segments/geophysical/tide_ocean'][:]
#                 data['tide_pole'] = fi[g+'/land_ice_segments/geophysical/tide_pole'][:]
                
#                rgt = fi['/orbit_info/rgt'][:]                           # single value
#                 t_ref = fi['/ancillary_data/atlas_sdp_gps_epoch'][:]     # single value
#                 beam_type = fi[g].attrs["atlas_beam_type"].decode()      # strong/weak (str)
#                 spot_number = fi[g].attrs["atlas_spot_number"].decode()  # number (str)
        
            # Assume all vector fields are of the length of the first. TODO - check?
            npts = len(data['lat'])
            data['ground_track'] = [g[1:]]*npts
                    
            # Make a dataframe out of our data dict and store it.
            dataframes.append(pd.DataFrame.from_dict(data))
        
    return dataframes

def atl06_2_df(files):
    """Return a single Pandas dataframe from a list of HDF5 ATL-06 data files.
    """
    dataframes = []
    for f in files:
        dataframes.extend(read_atl06(f))
    
    ndfs = len(dataframes)
    i = 0
    # pd.concat can only work with up to 10 dataframes at a time,
    # so we need to chunk this up
    new_dfs = []
    while i <= ndfs:
        i_end = i+10 if i+10 < ndfs else ndfs
        dfs = dataframes[i:i_end]
        if not dfs:
            break
        new_dfs.append(pd.concat(dfs))
        i = i_end
    
    return pd.concat(new_dfs)

In [102]:
path = Path('download')
dataf = atl06_2_df(path.glob('*.h5'))

Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt2l', '/gt2r', '/gt3l', '/gt3r']
Available ground tracks: ['/gt1l', '/gt1r', '/gt2l', '/gt2

In [104]:
dataf.describe()

,lat,lon,h_li,s_li,t_dt,q_flag
count,1.043409e+06,1.043409e+06,1.043409e+06,1.043409e+06,1.043409e+06,1.043409e+06
mean,5.991029e+01,-2.281352e+01,inf,inf,4.477354e+07,2.766528e-01
std,7.974897e+00,3.461218e+01,inf,inf,1.329375e+05,4.473435e-01
min,-3.226029e+00,-5.070999e+01,-2.403692e+02,2.701115e-03,4.459702e+07,0.000000e+00
25%,6.072619e+01,-4.490125e+01,1.015016e+03,1.954696e-02,4.465472e+07,0.000000e+00
50%,6.264569e+01,-4.351723e+01,1.792474e+03,4.119325e-02,4.473960e+07,0.000000e+00
75%,6.439309e+01,6.467029e+00,2.607161e+03,2.521526e-01,4.495368e+07,1.000000e+00
max,6.572999e+01,6.807996e+01,3.402823e+38,3.402823e+38,4.499985e+07,1.000000e+00


In [107]:
dataf.ground_track.value_counts()

gt3r    176640
gt3l    176609
gt2r    174064
gt2l    174024
gt1l    171038
gt1r    171034
Name: ground_track, dtype: int64

In [109]:
dataf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043409 entries, 0 to 37
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   ground_track  1043409 non-null  object 
 1   lat           1043409 non-null  float64
 2   lon           1043409 non-null  float64
 3   h_li          1043409 non-null  float32
 4   s_li          1043409 non-null  float32
 5   t_dt          1043409 non-null  float64
 6   q_flag        1043409 non-null  int8   
dtypes: float32(2), float64(3), int8(1), object(1)
memory usage: 48.8+ MB
